# 🧠 Hybrid Text Generator (Markov + GPT-2)
**Creative text generation using statistical and neural NLP methods.**
- Markov Chains (for structure & grammar)
- GPT-2 (for coherence & creativity)
- Gradio UI
- GPU-accelerated (Colab + HuggingFace)

In [ ]:
# ✅ Install required libraries
!pip install markovify gradio transformers textstat matplotlib

In [ ]:
# 📂 Upload your cleaned dataset files here (pride_sentences.txt, frankenstein_sentences.txt, etc.)
from google.colab import files
uploaded = files.upload()

In [ ]:
# 📚 Load Markovify models from cleaned text
import markovify

with open("pride_sentences.txt") as f:
    text1 = f.read()
with open("frankenstein_sentences.txt") as f:
    text2 = f.read()

model1 = markovify.Text(text1, state_size=3)
model2 = markovify.Text(text2, state_size=3)

combined_model = markovify.combine([model1, model2], [1, 1])

In [ ]:
# 🤖 Load GPT-2 Model (HuggingFace Transformers)
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

In [ ]:
# ✨ GPT-2 Text Generation Function
def refine_with_gpt2(prompt, max_new_tokens=100, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, 
                             temperature=temperature, top_p=top_p,
                             pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# 🚀 Gradio UI
import gradio as gr

def hybrid_generate(user_prompt, max_tokens, temp, top_p):
    markov_prompt = user_prompt.strip() if user_prompt else combined_model.make_short_sentence(120)
    gpt_output = refine_with_gpt2(markov_prompt, max_new_tokens=max_tokens, temperature=temp, top_p=top_p)
    return markov_prompt, gpt_output

demo = gr.Interface(
    fn=hybrid_generate,
    inputs=[
        gr.Textbox(label="Optional Prompt (Leave empty for random Markov)", lines=1),
        gr.Slider(50, 300, value=100, step=10, label="Max Tokens (GPT-2)"),
        gr.Slider(0.1, 1.5, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(0.5, 1.0, value=0.9, step=0.05, label="Top-p (Nucleus Sampling)")
    ],
    outputs=[
        gr.Textbox(label="Markov Prompt"),
        gr.Textbox(label="GPT-2 Refined Output")
    ],
    title="Hybrid Text Generator (Markov + GPT-2)",
    description="Generate creative text by combining statistical and neural NLP methods."
)

demo.launch(share=True)